In [10]:
!pip install git+https://github.com/dllllb/pytorch-lifestream.git@main
!pip install -U 'torch<2'
!pip install -U 'pytorch-lightning<2'
!pip install -U "torchvision<0.15.1"
!pip install duckdb

  Cloning https://github.com/dllllb/pytorch-lifestream.git (to revision main) to /tmp/pip-req-build-n7hngqfg
  Running command git clone --filter=blob:none --quiet https://github.com/dllllb/pytorch-lifestream.git /tmp/pip-req-build-n7hngqfg
  Resolved https://github.com/dllllb/pytorch-lifestream.git to commit b3d80211ec752c44ab7ece19830571b2f182b029
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [11]:
import pandas as pd
import numpy as np
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames.coles import CoLESModule
from tqdm.auto import tqdm, trange
from copy import deepcopy
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.frames.coles import ColesDataset
from catboost import CatBoostClassifier, Pool, cv
from ptls.preprocessing import PandasDataPreprocessor
from functools import partial
import random
import pytorch_lightning as pl
from ptls.data_load.datasets import inference_data_loader
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames import PtlsDataModule

In [12]:
import torch
import pytorch_lightning as pl
from functools import partial
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames.coles import CoLESModule
from ptls.frames.coles.losses import SoftmaxLoss
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.frames.coles import ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames import PtlsDataModule
from ptls.frames.inference_module import InferenceModule
from ptls.data_load.utils import collate_feature_dict

In [13]:
data = pd.read_csv('./dataset.csv',parse_dates=['client_start_date','partnerrolestart_date'])
simple_sub = pd.read_csv('./submission_example.csv')

In [14]:
def create_time_features(df):
    df['month'] = df['client_start_date'].dt.month
    df['day'] = df['client_start_date'].dt.day
    df['year'] = df['client_start_date'].dt.year

    df['dayofweek'] = df['client_start_date'].dt.dayofweek
    df['all_year_time'] = df['client_start_date'].apply(lambda x: (x.year - 2019) * 365 + x.month * 30 + x.day)
    df['cl_is_weekend'] = (df['dayofweek'] > 4).astype('int16')    
    return df

data = create_time_features(data)

In [15]:
train_data = data.copy()
train_data = train_data[train_data['client_start_date'] < pd.to_datetime('2020-9-01')]

train_data = train_data.sort_values(by='clientbankpartner_pin')
train_data['days_to_report'] = (pd.to_datetime('2020-9-01') - train_data['client_start_date']).dt.days

In [16]:
test_data = data.copy()

test_data = test_data.sort_values(by='clientbankpartner_pin')
test_data['days_to_report'] = (pd.to_datetime('2020-12-01') - test_data['client_start_date']).dt.days

In [17]:
preprocessor = PandasDataPreprocessor(
    col_id='clientbankpartner_pin',
    col_event_time='client_start_date',
    event_time_transformation='dt_to_timestamp',
    cols_category=None,
    cols_numerical=['days_to_report'],
)

train_data_proc = preprocessor.fit_transform(train_data[['clientbankpartner_pin','client_start_date','days_to_report']])
test_data_proc = preprocessor.fit_transform(test_data[['clientbankpartner_pin','client_start_date','days_to_report']])

In [18]:
trx_encoder = TrxEncoder(
    embeddings_noise=0.003,
    numeric_values={'days_to_report': 'identity'},
    #embeddings={'partner_src_type_ccode': {'in': 6, 'out': 1},}
)

# Sequence Encoder
seq_encoder = RnnSeqEncoder(
    trx_encoder=trx_encoder,
    hidden_size=32,  # Dimension of the generated embeddings
    type='gru',
)

# CoLES Module
coles_module = CoLESModule(seq_encoder=seq_encoder,
                           optimizer_partial=partial(torch.optim.AdamW, lr=0.001),
                           lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=5, gamma=0.9)
                          )

In [19]:
train_dl = PtlsDataModule(
    train_data=ColesDataset(
        MemoryMapDataset(
            data=train_data_proc,
            i_filters=[
                SeqLenFilter(min_seq_len=5),
            ],
        ),
        splitter=SampleSlices(
            split_count=5,
            cnt_min=5,
            cnt_max=200,
        ),
    ),
    train_num_workers=4,
    train_batch_size=128,
)

In [20]:
trainer = pl.Trainer(max_epochs=5, gpus=1)

trainer.fit(coles_module, train_dl)

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (26) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

In [21]:
inference_module = InferenceModule(
        model=seq_encoder,
        pandas_output=True,
        drop_seq_features=True,
        model_out_name=f'emb')

In [22]:
def TTA_Reverse_inferens(num_iters=10):
    tr_fold = []
    ts_fold = []
    for i in tqdm(range(num_iters)):
        train_data = []
        for t in train_data_proc:
            idx = torch.randperm(len(t['event_time']))
            train_data += [{'clientbankpartner_pin':t['clientbankpartner_pin'],
                            'event_time': t['event_time'][idx],
                            'days_to_report':t['days_to_report'][idx]}]
        
        test_data = []
        for t in test_data_proc:
            idx = torch.randperm(len(t['event_time']))
            test_data += [{'clientbankpartner_pin':t['clientbankpartner_pin'],
                            'event_time': t['event_time'][idx],
                            'days_to_report':t['days_to_report'][idx]}]
        
        inference_dataset_train = MemoryMapDataset(
            data=train_data,
            )

        inference_dl_train = torch.utils.data.DataLoader(
            dataset=inference_dataset_train,
            collate_fn=collate_feature_dict,
            shuffle=False,
            batch_size=128,
            num_workers=8,
            )

        inference_dataset_test = MemoryMapDataset(
            data=test_data,
            )

        inference_dl_test = torch.utils.data.DataLoader(
            dataset=inference_dataset_test,
            collate_fn=collate_feature_dict,
            shuffle=False,
            batch_size=128,
            num_workers=8,
            )
        predict = pl.Trainer(gpus=1).predict(inference_module, inference_dl_train)
        train_df = pd.concat(predict, axis=0).set_index('clientbankpartner_pin')
        predict = pl.Trainer(gpus=1).predict(inference_module, inference_dl_test)
        testdf = pd.concat(predict, axis=0).set_index('clientbankpartner_pin')
        
        tr_fold.append(train_df)
        ts_fold.append(testdf)
    return tr_fold, ts_fold

In [23]:
def clc_mean(lst):
    df_ref = lst[1]
    for i in range(2,len(lst)):
        df_ref += lst[i]
    df_ref.columns = [f'{x}_tta' for x in df_ref.columns]
    return df_ref / len(lst)

tr_fold, ts_fold = TTA_Reverse_inferens()
train_df_tta = clc_mean(tr_fold)
test_df_tta = clc_mean(ts_fold)

  0%|          | 0/10 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Predicting: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Predicting: 0it [00:00, ?it/s]

...

Predicting: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(


Predicting: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Predicting: 0it [00:00, ?it/s]

In [24]:
inference_dataset_train = MemoryMapDataset(
    data=train_data_proc,
)

inference_dl_train = torch.utils.data.DataLoader(
    dataset=inference_dataset_train,
    collate_fn=collate_feature_dict,
    shuffle=False,
    batch_size=128,
    num_workers=8,
)

inference_dataset_test = MemoryMapDataset(
    data=test_data_proc,
)

inference_dl_test = torch.utils.data.DataLoader(
    dataset=inference_dataset_test,
    collate_fn=collate_feature_dict,
    shuffle=False,
    batch_size=128,
    num_workers=8,
)

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [25]:
predict = pl.Trainer(gpus=1).predict(inference_module, inference_dl_train)
train_df = pd.concat(predict, axis=0).set_index('clientbankpartner_pin')

predict = pl.Trainer(gpus=1).predict(inference_module, inference_dl_test)
testdf = pd.concat(predict, axis=0).set_index('clientbankpartner_pin')

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(


Predicting: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Predicting: 0it [00:00, ?it/s]

In [26]:
train_df = pd.concat([train_df,train_df_tta],axis=1)
testdf = pd.concat([testdf,test_df_tta],axis=1)

In [27]:
def get_labels(data,treshold_date='2020-9-01'):
    date = pd.to_datetime(treshold_date)
    labels = data.groupby('clientbankpartner_pin')[['client_start_date']].agg('max')
    labels['score'] = labels['client_start_date'].apply(lambda x: 0 if x >= date else 1)
    labels['label_count'] = data.groupby('clientbankpartner_pin')['client_start_date'].agg(lambda x: len([i for i in x.tolist() if i >= date]) )
    return labels

labels = get_labels(data)

In [28]:
def get_val_splits(data,num_splits=10,n_val=1000,random_seed=56):
    date = pd.to_datetime('2020-6-01')
    labels = data.groupby('clientbankpartner_pin')[['client_start_date']].agg('max')
    good_partners = labels[labels['client_start_date'] >= date].index.tolist()
    all_partners = labels.index.tolist()
    train_idxs, val_idxs = [],[]
    for i in range(num_splits):
        random.seed(random_seed+i)
        random.shuffle(good_partners) # REFFF
        val_part = good_partners[:n_val]
        tr_part = [part for part in all_partners if part not in val_part]
        train_idxs.append(tr_part)
        val_idxs.append(val_part)
    return train_idxs, val_idxs

train_splits, eval_splits = get_val_splits(train_data)

In [29]:
params = {'iterations':2_000,
          'loss_function':'CrossEntropy',
          'learning_rate':0.01,
          'task_type':'CPU',
          'min_child_samples':20,
          'max_depth':5,
          'eval_metric':'AUC',
          'random_seed':56}

In [30]:
models = []
scores = []
for i in trange(len(train_splits)):
    train_pool = Pool(data=train_df.loc[train_splits[i]],
                      label=labels['score'].loc[train_splits[i]])
    eval_pool = Pool(data=train_df.loc[eval_splits[i]],
                     label=labels['score'].loc[eval_splits[i]])

    
    cbm = CatBoostClassifier(**params)
    cbm.fit(train_pool,eval_set=eval_pool,verbose=300)
    scores.append(cbm.get_best_score())
    models.append(cbm)

  0%|          | 0/10 [00:00<?, ?it/s]

0:	test: 0.7020471	best: 0.7020471 (0)	total: 64.6ms	remaining: 2m 9s
300:	test: 0.7649859	best: 0.7650498 (299)	total: 2.81s	remaining: 15.8s
600:	test: 0.7661099	best: 0.7664165 (569)	total: 5.46s	remaining: 12.7s
900:	test: 0.7660971	best: 0.7668124 (717)	total: 8.46s	remaining: 10.3s
1200:	test: 0.7663781	best: 0.7668124 (717)	total: 11.1s	remaining: 7.38s
1500:	test: 0.7655181	best: 0.7668124 (717)	total: 13.7s	remaining: 4.55s
1800:	test: 0.7634787	best: 0.7668124 (717)	total: 16.3s	remaining: 1.8s
1999:	test: 0.7620992	best: 0.7668124 (717)	total: 18s	remaining: 0us

bestTest = 0.7668124204
bestIteration = 717

Shrink model to first 718 iterations.
0:	test: 0.7321408	best: 0.7321408 (0)	total: 9.73ms	remaining: 19.4s
300:	test: 0.7964500	best: 0.7964500 (300)	total: 2.75s	remaining: 15.5s
600:	test: 0.7975824	best: 0.7977573 (507)	total: 5.41s	remaining: 12.6s
900:	test: 0.7959754	best: 0.7977573 (507)	total: 8.04s	remaining: 9.8s
1200:	test: 0.7920412	best: 0.7977573 (507)	tota

In [31]:
np.mean([x['validation']['AUC'] for x in scores])

0.782309244835566

In [32]:
test_df_lc = testdf.loc[simple_sub['clientbankpartner_pin']]
scores = np.mean([cbm.predict_proba(test_df_lc)[:,1] for cbm in models],axis=0)

In [33]:
simple_sub['score'] = scores

In [35]:
simple_sub

clientbankpartner_pin     score
0                      6781  0.048968
1                    236905  0.094670
2                    125779  0.230233
3                      1952  0.210518
4                      4872  0.572466
...                     ...       ...
4503                 121120  0.341221
4504                  60667  0.677298
4505                   5065  0.642821
4506                 133125  0.702593
4507                 121963  0.757668

[4508 rows x 2 columns]

In [36]:
simple_sub.to_csv('EmbedsTTA_Infer.csv',index=False)